In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Carga de los datos

In [2]:
file = "../Data/PSG1_0001.edf"
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()

Extracting EDF parameters from e:\Python\TFM\SSD_IA3\Data\PSG1_0001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<ipython-input-2-f4ce31b30072>:2: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(file)
<ipython-input-2-f4ce31b30072>:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(file)


In [9]:
events, events_id = mne.events_from_annotations(data)

Used Annotations descriptions: ['EEG arousal', 'Hypopnea', 'Lights Off', 'Limb Movement', 'Montage:PR, Ref', 'Obstructive Apnea', 'Oximeter Event', 'Oxygen Desaturation', 'Pulse Rate Event', 'Recording Analyzer - Auto-Staging', 'Recording Analyzer - Data Trends', 'Recording Analyzer - ECG', 'Recording Analyzer - Sleep Events', 'Sleep stage N1', 'Sleep stage N2', 'Sleep stage N3', 'Sleep stage R', 'Sleep stage W', 'Snoring', 'Start Recording', 'Started Analyzer - Sleep Events', 'Video Recording ON']


In [10]:
events.shape, events_id

((3014, 3),
 {'EEG arousal': 1,
  'Hypopnea': 2,
  'Lights Off': 3,
  'Limb Movement': 4,
  'Montage:PR, Ref': 5,
  'Obstructive Apnea': 6,
  'Oximeter Event': 7,
  'Oxygen Desaturation': 8,
  'Pulse Rate Event': 9,
  'Recording Analyzer - Auto-Staging': 10,
  'Recording Analyzer - Data Trends': 11,
  'Recording Analyzer - ECG': 12,
  'Recording Analyzer - Sleep Events': 13,
  'Sleep stage N1': 14,
  'Sleep stage N2': 15,
  'Sleep stage N3': 16,
  'Sleep stage R': 17,
  'Sleep stage W': 18,
  'Snoring': 19,
  'Start Recording': 20,
  'Started Analyzer - Sleep Events': 21,
  'Video Recording ON': 22})

In [11]:
tmax = 30. - 1. / data.info['sfreq']  # tmax in included

epochs_train = mne.Epochs(raw=data, events=events,
                          event_id=events_id, tmin=0., tmax=tmax, baseline=None, event_repeated='merge')

print(epochs_train)

Not setting metadata
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
3007 matching events found
No baseline correction applied
0 projection items activated
<Epochs |  3007 events (good & bad), 0 - 29.998 sec, baseline off, ~51 kB, data not loaded,
 'EEG arousal': 19
 'Hypopnea': 23
 'Lights Off': 1
 'Limb Movement': 100
 'Limb Movement/Snoring': 1
 'Montage:PR, Ref/Start Recording': 1
 'Obstructive Apnea': 79
 'Oximeter Event': 1
 'Oximeter Event/Pulse Rate Event': 1
 'Oxygen Desaturation': 47
 'Recording Analyzer - Auto-Staging/Recording Analyzer - Data Trends/Recording Analyzer - ECG/Recording Analyzer - Sleep Events': 1
 'Sleep stage N1': 25
 'Sleep stage N2': 317
 'Sleep stage N3': 193
 'Sleep stage R': 101
 'Sleep stage W': 226
 'Snoring': 1869
 'Started Analyzer - Sleep Events': 1
 'Video Recording ON': 1>


In [12]:
patatuela = epochs_train.load_data()

Loading data for 3007 events and 15360 original time points ...
1 bad epochs dropped


In [59]:
patatuela.events

array([[      19,        0,       23],
       [      67,        0,       22],
       [     101,        0,       24],
       ...,
       [13194240,        0,       18],
       [13209600,        0,       18],
       [13224960,        0,       18]])

In [32]:
patatuela.event_id

{'EEG arousal': 1,
 'Hypopnea': 2,
 'Lights Off': 3,
 'Limb Movement': 4,
 'Obstructive Apnea': 6,
 'Oximeter Event': 7,
 'Oxygen Desaturation': 8,
 'Sleep stage N1': 14,
 'Sleep stage N2': 15,
 'Sleep stage N3': 16,
 'Sleep stage R': 17,
 'Sleep stage W': 18,
 'Snoring': 19,
 'Started Analyzer - Sleep Events': 21,
 'Video Recording ON': 22,
 'Montage:PR, Ref/Start Recording': 23,
 'Recording Analyzer - Auto-Staging/Recording Analyzer - Data Trends/Recording Analyzer - ECG/Recording Analyzer - Sleep Events': 24,
 'Limb Movement/Snoring': 25,
 'Oximeter Event/Pulse Rate Event': 26}

In [43]:
id_to_class_dict = {value:key for key, value in patatuela.event_id.items()}
id_to_class_dict

{1: 'EEG arousal',
 2: 'Hypopnea',
 3: 'Lights Off',
 4: 'Limb Movement',
 6: 'Obstructive Apnea',
 7: 'Oximeter Event',
 8: 'Oxygen Desaturation',
 14: 'Sleep stage N1',
 15: 'Sleep stage N2',
 16: 'Sleep stage N3',
 17: 'Sleep stage R',
 18: 'Sleep stage W',
 19: 'Snoring',
 21: 'Started Analyzer - Sleep Events',
 22: 'Video Recording ON',
 23: 'Montage:PR, Ref/Start Recording',
 24: 'Recording Analyzer - Auto-Staging/Recording Analyzer - Data Trends/Recording Analyzer - ECG/Recording Analyzer - Sleep Events',
 25: 'Limb Movement/Snoring',
 26: 'Oximeter Event/Pulse Rate Event'}

In [52]:
patatuela._data.shape, patatuela.events

((3006, 50, 15360),
 array([[      19,        0,       23],
        [      67,        0,       22],
        [     101,        0,       24],
        ...,
        [13194240,        0,       18],
        [13209600,        0,       18],
        [13224960,        0,       18]]))

In [67]:
objective_stages = ['Sleep stage N1',
'Sleep stage N2',
'Sleep stage N3',
'Sleep stage R',
'Sleep stage W']

In [88]:
epochs_to_df = [a for a in patatuela]
label_to_df = [id_to_class_dict[i] for i in patatuela.events[:,-1]]
dict_to_df = {"Epoch":epochs_to_df,
              "Label":label_to_df}
# dict_to_df = {key:label for key, label in dict_to_df.items() if label in objective_stages}

In [ ]:
prueba = {}

In [93]:
epochs_to_df = []
label_to_df = []
for epoch, label in zip(patatuela, patatuela.events[:,-1]):
    # print(label)
    label = id_to_class_dict[label]
    if label in objective_stages:
        # print("True")
        epochs_to_df.append(epoch)
        label_to_df.append(label)
    # else:
        # print("False")

In [95]:
dict_to_df = {"Epoch":epochs_to_df,
              "Label":label_to_df}

In [96]:
pd.DataFrame(dict_to_df)

,Epoch,Label
0,"[[-0.0004177615014877544, -0.00041776150148775...",Sleep stage W
1,"[[-9.278095674067275e-05, -8.421149004348803e-...",Sleep stage W
2,"[[-0.00019297779812314013, -0.0001942961776150...",Sleep stage W
3,"[[-2.356603341725778e-05, -2.191805905241457e-...",Sleep stage W
4,"[[-4.2352941176470415e-05, -4.2352941176470415...",Sleep stage W
...,...,...
856,"[[2.3895628290226768e-05, 2.785076676585048e-0...",Sleep stage W
857,"[[2.191805905241491e-05, 2.5214007782101335e-0...",Sleep stage W
858,"[[1.0382238498512416e-05, 1.33485923552302e-05...",Sleep stage W
859,"[[4.7956054016937685e-05, 4.762645914396904e-0...",Sleep stage W
